# House price prediction



[![Open in Layer](https://development.layer.co/assets/badge.svg)](https://app.layer.ai/layer/california_housing) [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/layerai/examples/blob/main/housing/housing.ipynb) [![Layer Examples Github](https://badgen.net/badge/icon/github?icon=github&label)](https://github.com/layerai/examples/tree/main/housing)


In [ ]:
!pip install layer-sdk -qqq

In [ ]:
import layer
from layer.decorators import model, resources, pip_requirements, fabric, dataset
layer.login()

In [ ]:
layer.init("california_housing")

In [21]:
@pip_requirements(packages=["seaborn"])
@model("housing")
def train():
  import pandas as pd
  import seaborn as sns
  import matplotlib.pyplot as plt
  from sklearn.linear_model import LinearRegression
  from sklearn import metrics
  import numpy as np
  train = layer.get_dataset('layer/california_housing/datasets/train').to_pandas()
  test = layer.get_dataset('layer/california_housing/datasets/test').to_pandas()
  train.dropna(inplace=True)
  plt.figure(figsize=(12,6))
  sns.distplot(train['median_house_value'])
  layer.log({"median house value":plt.gcf()})
  corr = train.corr()
  plt.figure(figsize=(12,8))
  sns.heatmap(corr,annot=True)
  layer.log({"corr Heatmap": plt.gcf()})
  X_train = train.drop(['median_house_value'],axis=1)
  y_train = train['median_house_value']
  X_test = test.drop(['median_house_value'],axis=1)
  y_test = test['median_house_value']
  model = LinearRegression()
  model.fit(X_train,y_train)
  predictions = model.predict(X_test)
  predctions_df = pd.DataFrame(predictions,columns=['Predicted Price'])
  layer.log({"Predictions DataFrame":predctions_df.head(100)})
  plt.figure(figsize=(12,6))
  sns.distplot((y_test-predictions))
  layer.log({"Predictions":plt.gcf()})
  plt.figure(figsize=(12,6))
  sns.scatterplot(y_test,predictions)
  layer.log({"Predictions scatter plot":plt.gcf()})
  layer.log({ 'Mean Absolute Error': metrics.mean_absolute_error(y_test, predictions),
      'Mean Squarred Error': metrics.mean_squared_error(y_test, predictions),
      'Root Mean Squared Error': np.sqrt(metrics.mean_squared_error(y_test, predictions))})
  coefficient = model.coef_
  coef_df = pd.DataFrame([coefficient],columns=X_test.columns)
  layer.log({"coefficient": coef_df})
  return model

In [ ]:
train()

In [35]:
test = layer.get_dataset('layer/california_housing/datasets/train').to_pandas()
model = layer.get_model('layer/california_housing/models/housing').get_train()
x = test.drop('median_house_value', axis=1)
model.predict(x.head(1))

array([7128.08452517])